In [ ]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from sklearn import metrics
import matplotlib.ticker as mticker
from mpl_finance import candlestick2_ohlc
import sys

In [ ]:
#directory = '/Users/filippo/Developer/tesi/results/hSBM/highlyvariable_7tissues'
directory = '/Users/filippo/Developer/tesi/gtex/hsbm/oversampling_15tissue'

In [ ]:
L = 5

# topic size

In [ ]:
def plot_topic_size(directory,l):
    df_topics = pd.read_csv("%s/topsbm/topsbm_level_%d_topics.csv"%(directory,l))
    sizes = []
    for t in df_topics.columns:
        sizes.append(len(df_topics.loc[:,t].dropna()))
    bins = np.linspace(0.5, np.max(sizes)+0.5, int((np.max(sizes)+1)/(np.max(sizes)/5)))
    fig=plt.figure()
    plt.title("[%d topics, level: %d]"%(len(df_topics.columns),l))
    plt.hist(sizes, histtype='step', lw=2, bins=bins)
    plt.xlabel("topic size (# genes)", fontsize=16)
    plt.ylabel("# topic of that size", fontsize=16)
    plt.show()
    fig.savefig("%s/topic_size_level%d.png"%(directory,l))

In [ ]:
for l in range(1,L+1):
    plot_topic_size(directory,l)

## Topic O

In [ ]:
df_mv=pd.read_csv("%s/meanVariances.csv"%directory, index_col=[0])
#df_o = pd.read_csv("%s/O.dat"%directory, header=None)
#df_mv.insert(3, 'occurrence', df_o.values)
#df_mv['occurrence'] = df_mv['occurrence']
df_mv.head()

In [ ]:
def get_candles(directory, level, ax):
    df_topics = pd.read_csv("%s/topsbm/topsbm_level_%d_topics.csv"%(directory,level))
    candles = {
        'open': [],
        'high': [],
        'low': [],
        'close':[],
        'size':[]
    }
    for topic in df_topics.columns:
        subarr = df_mv.loc[df_topics[topic].dropna(),:]['occurrence'].values
        avg = np.average(subarr)
        std = np.std(subarr)
        q=np.quantile(subarr,[0.25,0.75])
        candles['high'].append(np.min([1,avg+std]))
        candles['open'].append(np.min([q[1],1]))
        candles['close'].append(np.max([q[0],0]))
        candles['low'].append(np.max([0,avg-std]))
        candles['size'].append(len(subarr))
    ax.set_title("[level: %d]"%(l))
    ax.set_ylabel('$O_i$', fontsize=18)
    ax.set_xlim(-1,len(df_topics.columns))
    ax.set_xticks([i+1 for i in range(-1,len(df_topics.columns))])
    ax.set_xticklabels(["Topic %d"%(i+2) if ((i+2)%5==0 or i==-1) else '' for i in range(-1,len(df_topics.columns))],  rotation=60)
    return candles

In [ ]:
for l in range(0,L+1):
    fig = plt.figure(figsize=(15,8))
    ax = fig.subplots(1,2)
    candles = get_candles(directory,l,ax[0])
    candlestick2_ohlc(ax[0], candles['open'],candles['high'],candles['low'],candles['close'],width=0.6,colordown='b')
    ax[1].hist((np.array(candles['open'])+np.array(candles['close']))/2, weights=candles['size'], range=(-0.05,1.05), bins=10, histtype='step')
    plt.show()
    fig.savefig("%s/topic_Ocandles_level_%d.pdf"%(directory,l))

# Geneontology

In [ ]:
from geneontology import get_ontology_df, ensg_to_symbol
from tableanalyser import get_symbol

In [ ]:
l=2
df_topics = pd.read_csv("%s/topsbm/topsbm_level_%d_topics.csv"%(directory,l))

In [ ]:
#import gseapy as gs
#gs.get_library_name()

In [ ]:
back_sea=[]
for topic in df_topics.columns:
    for g in df_topics[topic].dropna().values:
        try:
            back_sea.append(get_symbol(g))
        except:
            print("error %s"%g)

In [ ]:
for topic in df_topics.columns:
    try:
        enriched_topic = pd.read_csv("%s/gsea_level_%d_topic%s(%d).csv"%(directory,l,topic,len(symbols)), index_col=[0])
    except:
        symbols = ensg_to_symbol(df_topics.loc[:,topic].dropna().values)
        print(topic, " - " ,len(symbols))
        enriched_topic = get_ontology_df(symbols, background=back_sea).sort_values(by=['Adjusted P-value'], ascending=True)
        enriched_topic = enriched_topic.loc[enriched_topic.index.values[:20],:]
        enriched_topic.to_csv("%s/gsea_level_%d_topic%s(%d).csv"%(directory,l,topic,len(symbols)))
    print(enriched_topic)
    topic_pvalues.append(enriched_topic.loc[enriched_topic.index.values[0],'Adjusted P-value'])

In [ ]:
topic_pvalues = []
for topic in df_topics.columns:
    symbols = ensg_to_symbol(df_topics.loc[:,topic].dropna().values)
    try:
        enriched_topic = pd.read_csv("%s/gsea_level_%d_topic%s(%d).csv"%(directory,l,topic,len(symbols)))
        if len(enriched_topic.index) >0:
            p_val = np.sort(enriched_topic['Adjusted P-value'])[0]
            topic_pvalues.append(-np.log10(p_val))
    except:
        print("error", sys.exc_info()[0])

In [ ]:
fig = plt.figure()
x = np.arange(1,1+len(topic_pvalues))
plt.plot(x,np.sort(topic_pvalues)[::1])
plt.plot([-np.log10(0.01) for _ in x], label='1\%', c='g', ls='--')
plt.xlabel("Topic", fontsize=16)
plt.ylabel('-log(P-value)', fontsize=16)
#plt.ylim(0,0.055)
#plt.yscale('log')
fig.savefig("%s/pvaluescrosstopic(%d).png"%(directory,l))